In [17]:
from mqt.bench import get_benchmark
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [18]:
min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1

In [19]:
def create_circuit(num_qubits: int) -> QuantumCircuit:
    ae = AmplitudeEstimation(
        num_eval_qubits=num_qubits - 1,  # -1 because of the to be estimated qubit
    )
    problem = get_estimation_problem()

    qc = ae.construct_circuit(problem)
    qc.name = "ae"
    qc.measure_all()
    return qc

In [20]:
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="ae",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------ae with qubit ",i,"---------------")
    #print(qc.draw())
print("=================================================================================================")
print(circuits)

---------------ae with qubit  3 ---------------
---------------ae with qubit  4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x75ce595d3080>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x75ce5abd6900>)]


In [21]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()

jobs=[]

for qubit_number,circuit_object in circuits: 
    job = sampler.run([circuit_object], shots=1000)
    results = job.result()
    counts= results[0].data["meas"].get_counts()
    #print(f" > Counts: {results[0].data["meas"].get_counts()}")
    jobs.append((qubit_number , counts))
print(jobs)

[(3, {'010': 251, '100': 263, '000': 22, '011': 230, '110': 93, '001': 58, '111': 76, '101': 7}), (4, {'1011': 16, '1111': 206, '0111': 272, '0100': 229, '0011': 8, '1100': 189, '1110': 7, '1001': 2, '0010': 11, '0001': 3, '1000': 14, '1010': 22, '0000': 15, '0101': 1, '1101': 5})]


In [22]:
def hellinger_fidelity_with_expected(p, q):
    p_sum = sum(p.values())
    q_sum = sum(q.values())
    if q_sum == 0:
        print("ERROR: polarization_fidelity(), expected distribution is invalid, all counts equal to 0")
        return 0
    p_normed = {}
    for key, val in p.items():
        p_normed[key] = val/p_sum
    q_normed = {}
    for key, val in q.items():
        q_normed[key] = val/q_sum
    total = 0
    for key, val in p_normed.items():
        if key in q_normed.keys():
            total += (np.sqrt(val) - np.sqrt(q_normed[key]))**2
            del q_normed[key]
        else:
            total += val
    total += sum(q_normed.values())
    if total < 0:
        print("WARNING: using absolute value in fidelity calculation")
        total = abs(total)
        
    dist = np.sqrt(total)/np.sqrt(2)
    fidelity = (1-dist**2)**2

    return fidelity

In [23]:
from fidelity import *
#device_dist1=[3, {'011': 242, '100': 277, '001': 57, '010': 231, '110': 70, '111': 81, '000': 35, '101': 7}, (4, {'0111': 233, '0100': 255, '1100': 210, '1110': 5, '1111': 213, '1010': 23, '1000': 11, '0010': 4, '0000': 11, '0011': 7, '0101': 3, '1011': 13, '1101': 5, '1001': 6, '0110': 1})]
h_f = []
for qubit_number,count in jobs:

    ideal_dist = {}
    if True:  # balanced case
        non_zero_states = [format(i, '0{}b'.format(qubit_number)) for i in range(1, 2**qubit_number)]
        prob = 1 / len(non_zero_states)
        ideal_dist = {state: prob for state in non_zero_states}
    else:  # constant case
        ideal_dist = { '0'*qubit_number: 1.0 }
    hf=hellinger_fidelity_with_expected(counts,ideal_dist)
    h_f.append((qubit_number , hf))
    print(hf)
print(h_f)

0.0
0.5084446259221558
[(3, 0.0), (4, 0.5084446259221558)]


In [14]:
def hellinger_fidelity_with_expected(p, q):  # p:result distribution,passed as a counts  & q: the expected distribution
    p_sum, q_sum = sum(p.values()), sum(q.values())
    if q_sum == 0:
        print("ERROR: expected distribution invalid (all counts = 0)"); return 0
    
    p_norm = {k: v/p_sum for k, v in p.items()}
    q_norm = {k: v/q_sum for k, v in q.items()}
    total = sum((np.sqrt(p_norm[k]) - np.sqrt(q_norm.pop(k)))**2 if k in q_norm else v
                for k, v in p_norm.items()) + sum(q_norm.values())
    if total < 0:
        print("WARNING: abs value in fidelity calc"); total = abs(total)
    dist = np.sqrt(total)/np.sqrt(2)
    return (1 - dist**2)**2

def rescale_fidelity(f, floor, new_floor):
    r = (1-new_floor)/(1-floor) * (f - 1) + 1
    return max(0, min(1, r))

def polarization_fidelity(counts, correct_dist, thermal_dist=None): 
    '''     counts: the measurement outcomes after `num_shots` algorithm runs
                        correct_dist: the distribution we expect to get for the algorithm running perfectly
                        thermal_dist: optional distribution to pass in distribution from a uniform superposition over all states. If `None`: generated as 
                        uniform_dist` with the same qubits as in `counts`'''
    n = len(next(iter(correct_dist)))                            # get length of random key in correct_dist to find how many qubits measured
    counts = {k.zfill(n): v for k, v in counts.items()}          # ensure that all keys in counts are zero padded to this length
    hf = hellinger_fidelity_with_expected(counts, correct_dist)   # calculate hellinger fidelity between measured expectation values and correct distribution
    if n > 16: return {'fidelity': hf, 'hf_fidelity': hf}         # to limit cpu and memory utilization, skip noise correction if more than 16 measured qubits  & if not provided, generate thermal dist based on number of qubits
    if thermal_dist is None: thermal_dist = uniform_dist(n)      # set our fidelity rescaling value as the hellinger fidelity for a depolarized state
    floor = hellinger_fidelity_with_expected(thermal_dist, correct_dist)   # rescale fidelity result so uniform superposition (random guessing) returns fidelity
      
    # rescaled to 0 to provide a better measure of success of the algorithm (polarization)return {'fidelity': rescale_fidelity(hf, floor, 0), 'hf_fidelity': hf}


In [16]:
for qubit_number,counts
fidelity = hellinger_fidelity_with_expected(counts, correct_dist)

NameError: name 'correct_dist' is not defined